In [1]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer
os.environ["WANDB_DISABLED"] = "true"

/home/bintangkevin29/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-09 22:36:41.648391: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 22:36:41.948209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 22:36:42.837933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Tens

In [2]:
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

In [3]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor

image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    image_encoder_model, text_decode_model)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.8.crossattention.q_attn.weight', 'h.10.crossattention.q_attn.bias', 'h.1.ln_cross_attn.bias', 'h.5.crossattention.q_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.6.ln_cross_attn.bias', 'h.3.crossattention.c_attn.weight', 'h.8.crossattention.c_proj.weight', 'h.6.crossattention.c_proj.weight', 'h.8.crossattention.q_attn.bias', 'h.11.crossattention.c_proj.bias', 'h.3.crossattention.q_attn.weight', 'h.9.ln_cross_attn.bias', 'h.7.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.2.crossattention.q_attn.weight', 'h.0.crossattention.q_attn.weight', 'h.6.crossattention.c_attn.bias', 'h.9.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.bias', 'h.9.crossattention.c_attn.weight', 'h.11.crossattention.c_proj.weight', 'h.6.ln_cross_attn.weight', 'h.1.ln_cross_attn.weight', 'h.9.crossattention.q_attn.bias', 'h.11.crossattention.c_attn.bias', 'h.1.cro

In [4]:
# image feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
# text tokenizer
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

/home/bintangkevin29/miniconda3/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
# GPT2 only has bos/eos tokens but not decoder_start/pad tokens
tokenizer.pad_token = tokenizer.eos_token

# update the model config
model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [6]:
output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('vit-gpt-model/tokenizer_config.json',
 'vit-gpt-model/special_tokens_map.json',
 'vit-gpt-model/vocab.json',
 'vit-gpt-model/merges.txt',
 'vit-gpt-model/added_tokens.json',
 'vit-gpt-model/tokenizer.json')

In [14]:
import datasets
dsCaption = datasets.load_dataset("justram/COCO2014-Captions")
dsImages = datasets.load_dataset("justram/COCO2014-Images")


Found cached dataset parquet (/home/bintangkevin29/.cache/huggingface/datasets/justram___parquet/justram--COCO2014-Captions-7e4513f720fd85c5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 3/3 [00:00<00:00, 393.72it/s]


KeyboardInterrupt: 

In [13]:
print(ds['test'][0])

{'text_id': 770337, 'caption': 'A man with a red helmet on a small moped on a dirt road. '}
